In [1]:
import bisect

import pandas as pd
import numpy as np
import dask.dataframe as dd
import pycaret.classification as pc

from catboost import CatBoostClassifier, Pool, EShapCalcType, EFeaturesSelectionAlgorithm, cv
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold

# full

In [15]:
#df = dd.read_parquet('..\\full_data\\figma_gender\\full_gender_data.parquet').compute()
df = dd.read_parquet('..\\full_data\\figma_age\\full_age_data.parquet').compute()
#берем нужные
df = df.loc[:,['user_id']+df.columns[12:-6].to_list()]

# добавляем target, удаляем nan
df = df.merge(pd.read_parquet('..\\full_data\\public_train.pqt',columns=['user_id','age'])[['age','user_id']], on = 'user_id', how = 'inner')
df = df.loc[~(df['age'].isna()) & (df['age'] != 'NA')]
df['age'] = df['age'].astype('int8')

def age_bucket(x):
    return bisect.bisect_left([25,35,45,55,65], x)
df['age'] = df['age'].map(age_bucket)

display(df.head())
display(df.info())

,user_id,googleads.g.doubleclick.net,yandex.ru,i.ytimg.com,vk.com,avatars.mds.yandex.net,ad.mail.ru,yastatic.net,apple.com,instagram.com,...,bask_3cpe_type_cd,bask_4cpe_type_cd,bask_5cpe_type_cd,bask_0url_host,bask_1url_host,bask_2url_host,bask_3url_host,bask_4url_host,bask_5url_host,age
0,0,0.196891,0.031088,0.025907,0.041451,0.046632,0.031088,0.051813,0.0,0.000000,...,0.136947,0.069146,0.013774,0.134186,0.342372,0.281677,0.149741,0.076944,0.015080,1
1,1,0.072588,0.112703,0.040115,0.023878,0.104107,0.034384,0.081184,0.0,0.000000,...,0.136947,0.069146,0.013774,0.130193,0.341496,0.284690,0.151575,0.077067,0.014979,2
2,2,0.092457,0.046229,0.126521,0.141119,0.072993,0.038929,0.046229,0.0,0.000000,...,0.136947,0.069146,0.013774,0.144648,0.349024,0.276713,0.143092,0.072281,0.014242,0
3,3,0.021818,0.061818,0.007273,0.178182,0.094545,0.080000,0.069091,0.0,0.000000,...,0.136947,0.069146,0.013774,0.148331,0.353465,0.277041,0.139772,0.068262,0.013129,3
4,4,0.061776,0.030888,0.048906,0.083655,0.046332,0.061776,0.045045,0.0,0.003861,...,0.136947,0.069146,0.013774,0.145739,0.350527,0.276666,0.142050,0.070993,0.014025,1


<class 'pandas.core.frame.DataFrame'>
Int64Index: 269999 entries, 0 to 269998
Columns: 3015 entries, user_id to age
dtypes: float32(3001), float64(12), int32(1), int64(1)
memory usage: 3.0 GB


None

In [16]:
df

,user_id,googleads.g.doubleclick.net,yandex.ru,i.ytimg.com,vk.com,avatars.mds.yandex.net,ad.mail.ru,yastatic.net,apple.com,instagram.com,...,bask_3cpe_type_cd,bask_4cpe_type_cd,bask_5cpe_type_cd,bask_0url_host,bask_1url_host,bask_2url_host,bask_3url_host,bask_4url_host,bask_5url_host,age
0,0,0.196891,0.031088,0.025907,0.041451,0.046632,0.031088,0.051813,0.0,0.000000,...,0.136947,0.069146,0.013774,0.134186,0.342372,0.281677,0.149741,0.076944,0.015080,1
1,1,0.072588,0.112703,0.040115,0.023878,0.104107,0.034384,0.081184,0.0,0.000000,...,0.136947,0.069146,0.013774,0.130193,0.341496,0.284690,0.151575,0.077067,0.014979,2
2,2,0.092457,0.046229,0.126521,0.141119,0.072993,0.038929,0.046229,0.0,0.000000,...,0.136947,0.069146,0.013774,0.144648,0.349024,0.276713,0.143092,0.072281,0.014242,0
3,3,0.021818,0.061818,0.007273,0.178182,0.094545,0.080000,0.069091,0.0,0.000000,...,0.136947,0.069146,0.013774,0.148331,0.353465,0.277041,0.139772,0.068262,0.013129,3
4,4,0.061776,0.030888,0.048906,0.083655,0.046332,0.061776,0.045045,0.0,0.003861,...,0.136947,0.069146,0.013774,0.145739,0.350527,0.276666,0.142050,0.070993,0.014025,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269994,415308,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.136947,0.069146,0.013774,0.107228,0.325776,0.297336,0.165610,0.087173,0.016877,3
269995,415309,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.136947,0.069146,0.013774,0.107228,0.325776,0.297336,0.165610,0.087173,0.016877,1
269996,415311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.136947,0.069146,0.013774,0.157811,0.343646,0.280284,0.128500,0.069439,0.020320,2
269997,415312,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.136947,0.069146,0.013774,0.129522,0.336409,0.281654,0.154609,0.081594,0.016212,2


In [4]:
display(df.isna().sum().sum())
for name in df['cpe_manufacturer_name'].unique():
    df.loc[(df['cpe_manufacturer_name']==name)&(df['price'].isna()),'price'] = \
        df.loc[(df['cpe_manufacturer_name']==name)&(~df['price'].isna()),'price'].median()
df.loc[df['price']<1000,'price'] = 1000
display(df.isna().sum().sum())

6949

221

In [17]:
x_train = df.drop(['age','user_id'],axis=1)
y_train = df['age']

pool_train = Pool(x_train, y_train,
                  cat_features = list(x_train.select_dtypes(include=['object']).columns))

model = CatBoostClassifier(random_state=42,
                           task_type='GPU')
model.fit(pool_train, verbose=100)

print(classification_report(y_train, model.predict(x_train), \
                            target_names = ['18-25','25-34', '35-44', '45-54', '55-65', '65+']))

Learning rate set to 0.203367
0:	learn: 1.6656930	total: 241ms	remaining: 4m 1s
100:	learn: 1.2629314	total: 16.2s	remaining: 2m 24s
200:	learn: 1.2322306	total: 30.4s	remaining: 2m
300:	learn: 1.2129979	total: 44.4s	remaining: 1m 43s
400:	learn: 1.1976423	total: 58.3s	remaining: 1m 27s
500:	learn: 1.1845253	total: 1m 12s	remaining: 1m 11s
600:	learn: 1.1742968	total: 1m 27s	remaining: 57.9s
700:	learn: 1.1645888	total: 1m 40s	remaining: 42.8s
800:	learn: 1.1552675	total: 1m 53s	remaining: 28.3s
900:	learn: 1.1471470	total: 2m 7s	remaining: 14s
999:	learn: 1.1398305	total: 2m 20s	remaining: 0us
              precision    recall  f1-score   support

       18-25       0.64      0.46      0.53     33718
       25-34       0.56      0.69      0.62     87270
       35-44       0.48      0.60      0.54     77486
       45-54       0.51      0.34      0.41     42442
       55-65       0.54      0.31      0.40     23580
         65+       0.76      0.07      0.13      5503

    accuracy      

## test

In [19]:
df = dd.read_parquet('..\\full_data\\figma_age\\full_age_data.parquet').compute()

#берем нужные
df = df.loc[:,['user_id']+df.columns[12:-6].to_list()]

# добавляем target, удаляем nan
submit = dd.read_parquet('..\\full_data\\submit_2.pqt').compute()
submit = submit.sort_values('user_id', ignore_index=True)
test_df = submit.merge(df, on='user_id',how ='left')

display(df.head())
display(df.info())

,user_id,googleads.g.doubleclick.net,yandex.ru,i.ytimg.com,vk.com,avatars.mds.yandex.net,ad.mail.ru,yastatic.net,apple.com,instagram.com,...,bask_2cpe_type_cd,bask_3cpe_type_cd,bask_4cpe_type_cd,bask_5cpe_type_cd,bask_0url_host,bask_1url_host,bask_2url_host,bask_3url_host,bask_4url_host,bask_5url_host
0,0,0.196891,0.031088,0.025907,0.041451,0.046632,0.031088,0.051813,0.0,0.000000,...,0.268126,0.136947,0.069146,0.013774,0.134186,0.342372,0.281677,0.149741,0.076944,0.015080
1,1,0.072588,0.112703,0.040115,0.023878,0.104107,0.034384,0.081184,0.0,0.000000,...,0.268126,0.136947,0.069146,0.013774,0.130193,0.341496,0.284690,0.151575,0.077067,0.014979
2,2,0.092457,0.046229,0.126521,0.141119,0.072993,0.038929,0.046229,0.0,0.000000,...,0.268126,0.136947,0.069146,0.013774,0.144648,0.349024,0.276713,0.143092,0.072281,0.014242
3,3,0.021818,0.061818,0.007273,0.178182,0.094545,0.080000,0.069091,0.0,0.000000,...,0.268126,0.136947,0.069146,0.013774,0.148331,0.353465,0.277041,0.139772,0.068262,0.013129
4,4,0.061776,0.030888,0.048906,0.083655,0.046332,0.061776,0.045045,0.0,0.003861,...,0.268126,0.136947,0.069146,0.013774,0.145739,0.350527,0.276666,0.142050,0.070993,0.014025


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 414723 entries, 0 to 414722
Columns: 3014 entries, user_id to bask_5url_host
dtypes: float32(3001), float64(12), int32(1)
memory usage: 4.7 GB


None

In [7]:
display(test_df.isna().sum().sum())
for name in df['cpe_manufacturer_name'].unique():
    test_df.loc[(test_df['cpe_manufacturer_name']==name)&(test_df['price'].isna()),'price'] = \
        df.loc[(df['cpe_manufacturer_name']==name)&(~df['price'].isna()),'price'].median()
test_df.loc[test_df['price']<1000,'price'] = 1000
display(test_df.isna().sum().sum())

3801

105

In [26]:
submit['age'] = model.predict(test_df.drop('user_id',axis=1))
submit['age'] += 1
submit['is_male'] = pd.Series(data=-1.0,index=submit.index)

In [27]:
submit.to_csv('age_urls_only.csv', index=False)